# Lineare Regression für das Wirtschaftpoltische Seminar 


- Ziel ein Lineare Regression  erstellen vom Europäische Euro Raum

## Daten für die Hausarbeit
- Euro Raum

Regression Model OLS
Model:

    - BIP = Oil_price + (m1,m2,m3) + CPI + evtl Arbeitslosigkeit

In [29]:
import pandas as pd
import datetime as dt

### BIP der Euro Raum erhalten von der Offizielen Seite 
- https://data.ecb.europa.eu/data/datasets/MNA/MNA.A.N.I9.W2.S1.S1.B.B1GQ._Z._Z._Z.PE_R_POP.V._Z 

In [30]:
BIPEUArea = pd.read_csv('/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/models/Linear Regression/Data_EU/ECB Data Portal_BIP_EuroArea.csv')
BIPEUArea.head()

,DATE,TIME PERIOD,Gross domestic product at market prices (MNA.A.N.I9.W2.S1.S1.B.B1GQ._Z._Z._Z.PE_R_POP.V._Z)
0,1995-12-31,1995,17927.30
1,1996-12-31,1996,18652.69
2,1997-12-31,1997,19069.85
3,1998-12-31,1998,19814.33
4,1999-12-31,1999,20671.06


In [31]:
BIPEUArea = BIPEUArea.drop(columns=['DATE'])
BIPEUArea = BIPEUArea.rename(columns={'TIME PERIOD': 'Date', 'Gross domestic product at market prices (MNA.A.N.I9.W2.S1.S1.B.B1GQ._Z._Z._Z.PE_R_POP.V._Z)': 'BIP'})
#BIP_EUArea['Date'] = pd.to_datetime(BIP_EUArea['Date'])

In [32]:
BIPEUArea.head()

,Date,BIP
0,1995,17927.30
1,1996,18652.69
2,1997,19069.85
3,1998,19814.33
4,1999,20671.06


## M1, M2, M3 Datasets

In [34]:
df_m1 = pd.read_csv('/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/models/Linear Regression/Data_EU/ECB Data M1 Euro_Area.csv')
df_m2 = pd.read_csv('/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/models/Linear Regression/Data_EU/ECB Data M2 Euro_Area.csv')
df_m3 = pd.read_csv('/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/models/Linear Regression/Data_EU/ECB Data M3 Euro_Area.csv')
# in Euro

In [36]:
df_m1.rename(columns={'Monetary aggregate M1 reported by MFIs, central gov. and post office giro institutions in the euro area (stocks) (BSI.M.U2.Y.V.M10.X.1.U2.2300.Z01.E)': 'M1',
                      'DATE': 'Date'}, inplace=True)
df_m2.rename(columns={'Monetary aggregate M2 reported by MFIs, central gov. and post office giro institutions in the euro area (stocks) (BSI.M.U2.Y.V.M20.X.1.U2.2300.Z01.E)': 'M2',
                      'DATE': 'Date'}, inplace=True)
df_m3.rename(columns={'Monetary aggregate M3 reported by MFIs, central gov. and post office giro institutions in the euro area (stocks) (BSI.M.U2.Y.V.M30.X.1.U2.2300.Z01.E)': 'M3',
                      'DATE': 'Date'}, inplace=True)

df_m1.drop(columns=['TIME PERIOD'], inplace=True)
df_m2.drop(columns=['TIME PERIOD'], inplace=True)
df_m3.drop(columns=['TIME PERIOD'], inplace=True)

In [37]:
#Change dateformat from YYYY-MM-DD to YYYY-MM (e.g. 2020-01-01 to 2020-01)
df_m1['Date'] = pd.to_datetime(df_m1['Date'])
df_m1['Date'] = df_m1['Date'].dt.to_period('M')

df_m2['Date'] = pd.to_datetime(df_m2['Date'])
df_m2['Date'] = df_m2['Date'].dt.to_period('M')

df_m3['Date'] = pd.to_datetime(df_m3['Date'])
df_m3['Date'] = df_m3['Date'].dt.to_period('M')

In [39]:
df_m3.head(), df_m2.head(), df_m1.head()

(      Date       M3
 0  1980-01  1097404
 1  1980-02  1105520
 2  1980-03  1117228
 3  1980-04  1122639
 4  1980-05  1130672,
       Date       M2
 0  1980-01  1070496
 1  1980-02  1078232
 2  1980-03  1088333
 3  1980-04  1093263
 4  1980-05  1100958,
       Date      M1
 0  1980-01  444330
 1  1980-02  447116
 2  1980-03  451370
 3  1980-04  453226
 4  1980-05  455917)

### Da wir Monatlich Daten haben und wir nun die Jährliche Daten haben möchten, müssen wir die Daten gruppieren.

In [65]:
import pandas as pd

# Assuming your CSV files have a 'Date' or 'Year-Month' column for timestamps
def convert_to_yearly(df):
  """Converts a pandas DataFrame with monthly data to a DataFrame with yearly data.

  Args:
      df (pd.DataFrame): The DataFrame containing monthly data.

  Returns:
      pd.DataFrame: A new DataFrame with yearly data.
  """

  # Convert 'Date' column to datetime if necessary
  if pd.api.types.is_period_dtype(df['Date']):
    df['Date'] = df['Date'].dt.to_timestamp()
  df['Year'] = pd.to_datetime(df['Date']).dt.year  # Assuming 'Date' column exists

  # Group by year and calculate the desired aggregation (e.g., sum, mean)
  if 'M1' in df.columns: 
    yearly_data = df.groupby('Year').agg(Sum=('M1', 'sum'))  # Replace 'column_to_sum' with your column
  elif 'M2' in df.columns: 
    yearly_data = df.groupby('Year').agg(Sum=('M2', 'sum'))
  elif 'M3' in df.columns: 
    yearly_data = df.groupby('Year').agg(Sum=('M3', 'sum'))

  return yearly_data

# Read the CSV files (replace paths if necessary)
df_m1_yearly = convert_to_yearly(df_m1.copy())
df_m2_yearly = convert_to_yearly(df_m2.copy())  
df_m3_yearly = convert_to_yearly(df_m3.copy())  

# Print or use the yearly DataFrames as needed
df_m1_yearly.reset_index(inplace=True)
df_m2_yearly.reset_index(inplace=True)
df_m3_yearly.reset_index(inplace=True)
df_m1_yearly.rename(columns={'Sum': 'M1', "Year": "Date"}, inplace=True)
df_m2_yearly.rename(columns={'Sum': 'M2', "Year": "Date"}, inplace=True)
df_m3_yearly.rename(columns={'Sum': 'M3', "Year": "Date"}, inplace=True)






/var/folders/13/3pzxxwtd6cbdvdzq_lc6112r0000gn/T/ipykernel_15293/327483836.py:15: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df['Date']):
/var/folders/13/3pzxxwtd6cbdvdzq_lc6112r0000gn/T/ipykernel_15293/327483836.py:15: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df['Date']):
/var/folders/13/3pzxxwtd6cbdvdzq_lc6112r0000gn/T/ipykernel_15293/327483836.py:15: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df['Date']):


In [53]:
BIPEUArea['Date'].min(), BIPEUArea['Date'].max(),

(1995, 2022)

### Filtern der Daten damit wir die gleichen Zeitraum haben.

In [71]:
df_m1_yearly_filtered = df_m1_yearly.query('Date >= 1995 & Date <= 2022')
df_m2_yearly_filtered = df_m2_yearly.query('Date >= 1995 & Date <= 2022')
df_m3_yearly_filtered = df_m3_yearly.query('Date >= 1995 & Date <= 2022')

print(df_m1_yearly_filtered.shape, df_m2_yearly_filtered.shape, df_m3_yearly_filtered.shape, BIPEUArea.shape)
print('M1', df_m1_yearly_filtered['Date'].min(), df_m1_yearly_filtered['Date'].max())
print('M2',df_m2_yearly_filtered['Date'].min(), df_m2_yearly_filtered['Date'].max())
print('M3', df_m3_yearly_filtered['Date'].min(), df_m3_yearly_filtered['Date'].max())
print('BIPEUArea', BIPEUArea['Date'].min(), BIPEUArea['Date'].max())

(28, 2) (28, 2) (28, 2) (28, 2)
M1 1995 2022
M2 1995 2022
M3 1995 2022
BIPEUArea 1995 2022


# M1, M2, M3, BIPEUArea wurden richtig fomratiert und gefiltert
- weiter geht es mit Oil Prices

## Oil Price Datasets
Herunterladen druch die FED Api

In [106]:
api_key = 'd9ce385a3f807aaba9590769b5f994a1'
OIL_dataURL = f'https://api.stlouisfed.org/fred/series/observations?series_id=POILBREUSDM&api_key={api_key}&file_type=json&frequency=m'
CPI_dataURL = f'https://api.stlouisfed.org/fred/series/observations?series_id=CPHPTT01EZM659N&api_key={api_key}&file_type=json&frequency=m'
un_dataURL = f'https://api.stlouisfed.org/fred/series/observations?series_id=LRHUTTTTEZM156S&api_key={api_key}&file_type=json&frequency=m'


In [113]:
import requests
import json
import pandas as pd

# Setzen Sie Ihren API-Schlüssel hier ein
api_key = 'd9ce385a3f807aaba9590769b5f994a1'
def get_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        observations = []
        for observation in data['observations']:
            date = observation['date']
            value = observation['value']
            if value != '.':
                value = float(value)
                observations.append({'Date': date, 'Value': value})
        df = pd.DataFrame(observations)
        return df
    else:
        print(f'Error: {response.status_code}')
        return None

Oil_price = get_data(OIL_dataURL)
CPI = get_data(CPI_dataURL)
unemployment = get_data(un_dataURL)
unemployment.shape, CPI.shape, Oil_price.shape

((391, 2), (385, 2), (410, 2))

In [108]:
Oil_price.columns, CPI.columns, unemployment.columns

(Index(['Date', 'Value'], dtype='object'),
 Index(['Date', 'Value'], dtype='object'),
 Index(['Date', 'Value'], dtype='object'))

In [114]:
def convert_to_yearly_(df):
  """Converts a pandas DataFrame with monthly data to a DataFrame with yearly data.

  Args:
      df (pd.DataFrame): The DataFrame containing monthly data.

  Returns:
      pd.DataFrame: A new DataFrame with yearly data.
  """

  # Convert 'Date' column to datetime if necessary
  if pd.api.types.is_period_dtype(df['Date']):
    df['Date'] = df['Date'].dt.to_timestamp()
  df['Date'] = pd.to_datetime(df['Date']).dt.year  # Assuming 'Date' column exists

  # Group by year and calculate the desired aggregation (e.g., sum, mean)
  if 'Date' in df.columns: 
    yearly_data = df.groupby('Date').agg(Sum=('Value', 'sum'))  # Replace 'column_to_sum' with your column
    yearly_data.reset_index(inplace=True)
  return yearly_data

Oil_price = convert_to_yearly_(Oil_price)
Oil_price

/var/folders/13/3pzxxwtd6cbdvdzq_lc6112r0000gn/T/ipykernel_15293/1190788173.py:12: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df['Date']):


,Date,Sum
0,1990,284.479368
1,1991,240.904445
2,1992,232.298854
3,1993,205.017111
4,1994,191.515897
5,1995,206.384976
6,1996,249.702360
7,1997,231.723884
8,1998,157.514630
9,1999,217.298631


In [119]:
Oil_price.rename(columns={'Sum': 'Oil_Price'}, inplace=True)
Oil_price_yearly_filtered = Oil_price.query('Date >= 1995 & Date <= 2022')
Oil_price_yearly_filtered.merge(df_merged, on='Date', how='outer').to_csv('/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/models/Linear Regression/Data_EU/ECB Data Merged Euro_Area.csv', index=False)

In [ ]:
Oil_price_yearly_filtered.head()


# Oil Price ende 

# Unemployment rate 

In [127]:
unemployment_rate_EUArea = get_data(un_dataURL)
unemployment_rate_EUArea = convert_to_yearly_(unemployment_rate_EUArea)
unemployment_rate_EUArea.rename(columns={'Sum': 'Unemployment_Rate'}, inplace=True)
unemployment_rate_EUArea_yearly_filtered = unemployment_rate_EUArea.query('Date >= 1995 & Date <= 2022')


/var/folders/13/3pzxxwtd6cbdvdzq_lc6112r0000gn/T/ipykernel_15293/1190788173.py:12: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df['Date']):


In [128]:
CPI_EUArea = get_data(CPI_dataURL)
CPI_EUArea = convert_to_yearly_(CPI_EUArea)
CPI_EUArea.rename(columns={'Sum': 'CPI'}, inplace=True)
CPI_EUArea_yearly_filtered = CPI_EUArea.query('Date >= 1995 & Date <= 2022')


/var/folders/13/3pzxxwtd6cbdvdzq_lc6112r0000gn/T/ipykernel_15293/1190788173.py:12: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df['Date']):


In [129]:
CPI_EUArea_yearly_filtered.shape, unemployment_rate_EUArea_yearly_filtered.shape, Oil_price_yearly_filtered.shape

((28, 2), (28, 2), (28, 2))

In [130]:
print(CPI_EUArea_yearly_filtered['Date'].min(), CPI_EUArea_yearly_filtered['Date'].max())
print(unemployment_rate_EUArea_yearly_filtered['Date'].min(), unemployment_rate_EUArea_yearly_filtered['Date'].max())
print(Oil_price_yearly_filtered['Date'].min(), Oil_price_yearly_filtered['Date'].max())



1995 2022
1995 2022
1995 2022


## Data Collection

## Creating a Regression Model

In [ ]:
import statsmodels.api as sm
import numpy as np
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.params